In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

In [3]:
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

dataset = load_dataset("Isamu136/big-animal-dataset")

captions = dataset['train']['caption']

README.md:   0%|          | 0.00/554 [00:00<?, ?B/s]

data/train-00000-of-00003-b0e3a3345037ee(…):   0%|          | 0.00/525M [00:00<?, ?B/s]

data/train-00001-of-00003-32ac7a4ee78dc9(…):   0%|          | 0.00/502M [00:00<?, ?B/s]

data/train-00002-of-00003-e455e1e1c98858(…):   0%|          | 0.00/781M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62149 [00:00<?, ? examples/s]

In [4]:
le = LabelEncoder()
caption_ids = le.fit_transform(captions)
class_names = list(le.classes_)

In [5]:
dataset['train'] = dataset['train'].add_column('lable', caption_ids)
dataset['train'] = dataset['train'].remove_columns("caption")

In [6]:
# from datasets import load_dataset
# from sklearn.preprocessing import LabelEncoder

# dataset = load_dataset("Isamu136/big-animal-dataset")

# captions = dataset['train']['caption']
# le = LabelEncoder()
# caption_ids = le.fit_transform(captions)
# dataset['train'] = dataset['train'].add_column('lable', caption_ids)
# dataset['train'] = dataset['train'].remove_columns("caption")

In [7]:
dataset_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_valid = dataset_split["train"].train_test_split(test_size=0.25, seed=42)
dataset = {
    "train": train_valid["train"],
    "validation": train_valid["test"],
    "test": dataset_split["test"]
}

In [8]:
# dataset['train'][1000]

In [9]:
from torch.utils.data import Dataset

class ToRGB(object):
    def __call__(self, img):
        return img.convert('RGB')

transform_train = transforms.Compose([
    ToRGB(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    ToRGB(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_validation = transforms.Compose([
    ToRGB(),
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

class AnimalDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        caption_id = self.dataset[idx]['lable']
        if self.transform:
            image = self.transform(image)
        return image, caption_id

train_dataset = AnimalDataset(dataset['train'], transform=transform_train)
test_dataset  = AnimalDataset(dataset['test'], transform=transform_test)
validation_dataset = AnimalDataset(dataset['validation'], transform=transform_test)


In [10]:
# train_dataset[0]

In [28]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=0):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)        
        self.fc = nn.Linear(512, num_classes)
        
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
lenght = len(class_names)

def ResNet18():
    return ResNet(ResidualBlock, num_classes = lenght)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18().to(device)

In [25]:
state_dict = torch.load('/kaggle/input/resnet2/cifar100_best_model_resnet18_seed2023.pth')


model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['conv1.0.weight', 'conv1.1.weight', 'conv1.1.bias', 'conv1.1.running_mean', 'conv1.1.running_var', 'layer1.0.left.0.weight', 'layer1.0.left.1.weight', 'layer1.0.left.1.bias', 'layer1.0.left.1.running_mean', 'layer1.0.left.1.running_var', 'layer1.0.left.3.weight', 'layer1.0.left.4.weight', 'layer1.0.left.4.bias', 'layer1.0.left.4.running_mean', 'layer1.0.left.4.running_var', 'layer1.1.left.0.weight', 'layer1.1.left.1.weight', 'layer1.1.left.1.bias', 'layer1.1.left.1.running_mean', 'layer1.1.left.1.running_var', 'layer1.1.left.3.weight', 'layer1.1.left.4.weight', 'layer1.1.left.4.bias', 'layer1.1.left.4.running_mean', 'layer1.1.left.4.running_var', 'layer2.0.left.0.weight', 'layer2.0.left.1.weight', 'layer2.0.left.1.bias', 'layer2.0.left.1.running_mean', 'layer2.0.left.1.running_var', 'layer2.0.left.3.weight', 'layer2.0.left.4.weight', 'layer2.0.left.4.bias', 'layer2.0.left.4.running_mean', 'layer2.0.left.4.running_var', 'layer2.0.shortcut.0.weight', 'laye

In [29]:
from torch.optim.lr_scheduler import OneCycleLR

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = OneCycleLR(
    optimizer,
    max_lr=1e-3,
    steps_per_epoch=len(train_dataloader),
    epochs=50
)

In [30]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [31]:
def train(train_loader, val_loader, model, loss_fn, optimizer, scheduler):
    early_stopping = EarlyStopping(patience=7, min_delta=1e-4)
    model.train()
    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step() # Step scheduler per batch!

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            pred_val = model(X_val)
            val_loss += loss_fn(pred_val, y_val).item()
    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.6f}")

    # Check early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        return True 

In [32]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [33]:
def validation(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
epochs = 70
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    stop = train(train_dataloader, validation_dataloader, model, loss_fn, optimizer, scheduler)
    test(test_dataloader, model, loss_fn)
    if stop:
        break
print("Done!")

Epoch 1
-------------------------------
loss: 3.262585  [   64/37289]
loss: 3.246325  [ 6464/37289]
loss: 3.193447  [12864/37289]
loss: 3.351963  [19264/37289]
loss: 3.750714  [25664/37289]
loss: 3.663683  [32064/37289]
Validation Loss: 4.418932
Test Error: 
 Accuracy: 12.6%, Avg loss: 4.455565 

Epoch 2
-------------------------------
loss: 3.399457  [   64/37289]
loss: 3.252271  [ 6464/37289]
loss: 3.226799  [12864/37289]
loss: 3.463130  [19264/37289]
loss: 3.373545  [25664/37289]
loss: 3.538735  [32064/37289]
Validation Loss: 4.723850
Test Error: 
 Accuracy: 9.5%, Avg loss: 4.761127 

Epoch 3
-------------------------------
loss: 3.414966  [   64/37289]


KeyboardInterrupt: 